# Asynchronous execution

 DPsim integrates well with event loop implementation like [asyncio](https://docs.python.org/3/library/asyncio.html).
 
 This allows the user to run simulations as a coroutine asynchronously in the background.

## Example 1

We start by defining a very simple simulation:

In [ ]:
import time
import asyncio
import dpsim
from dpsim.EventChannel import Event

# Nodes
gnd = dpsim.dp.Node.GND()
n1 = dpsim.dp.Node("n1")

# Components
v1 = dpsim.dp.ph1.VoltageSource("v_1", [gnd, n1], V_ref=complex(345,0))

sys = dpsim.SystemTopology(50, [gnd, n1], [v1])

The <code>dpsim.Simulation</code> class has a function called `coro simulate()` which returns a coroutine.
this co-routine can be started in the background via:

In [ ]:
sims = []
for i in range(1, 4):
    sim = dpsim.RealTimeSimulation("async_demo_%d" % i, sys, timestep=i*1e-3, duration=3*i+5, pbar=True)
    sim.start()
    
    sims += [sim]

for i in range(1, 6):
    print("Doing something different: %d" % i)
    await asyncio.sleep(1)
    
_ = await asyncio.wait([ s.wait(Event.done) for s in sims ])